# Non-Standard Evaluation

One of R's greatest strengths comes from its non-standard approach to code evaluation, or NSE for short. In most programming languages the programmer has access to the values of a function's arguments, but not the code the code used to compute them. However, in R it is possible to access the code directly, which opens up a plethora of opportunities for evaluating that code in non-standard ways. This ability can be put to good use when creating functions for doing interactive data exploration because it can dramatically reduce the amount of typing done by the analyst.

We'll get into the details of how this works in a bit, but before we can dive into the particulars of non-standard evaluation, we need to take a look at another of R's features: lazy evaluation. 

## Lazy Evaluation

In R, all function arguments are evaluated only on the first time they are accessed. To accomplish this, R wraps all arguments in a special data type called a promise that captures both the expression needed to compute the argument's value and the environment in which to compute it. This allows us to play things a bit fast and loose with our functions like the example below demonstrates.

In [20]:
dbl1 <- function(x, y) { x * 2 }
dbl1(4)
dbl1(4, nonExistentObject)
nonExistentObject

dbl2 <- function(x, y) { x * 2; force(y) }
dbl2(4)

[1] 8

[1] 8

ERROR: Error in eval(expr, envir, enclos): object 'nonExistentObject' not found


Notice that we can call the first function, `dbl1`, without passing in a value for the `y` argument without getting an error. The reason that we don't get an argument missing error is because the `y` argument is never used within the function, and since arguments are lazily evaluated, we don't see an error. However, in the second function, `dbl2`, we call the `force` function to force the evaluation of the `y` argument. In this case, since we are evaluating `y`, we get an error that our function call is missing a value for an argument.

## Non-standard Evaluation

As mentioned above, and as part of R's lazy evaluation mechanism, arguments in R are wrapped in a data structure that preserves the code used to produce the argument's value. This implementation detail also gives developers the ability to capture these expressions and operate on them in non-standard ways. Using the `substitute`, `quote`, `deparse`, and `eval` functions allows us implement some really interesting behavior in our functions. 

<!-- More specifically, the `substitute` and `quote` functions allow us to capture each argument's expression, and the `deparse` and `eval` functions allow us to capture the code itself and evaluate the code within a new context respectively. -->

In [14]:
f <- function(x) {
    substitute(x)
}
args <- f(1:10)

In [18]:
class(args)

[1] "call"

In [16]:
deparse(args)

[1] "1:10"

In [22]:
avg <- quote(mean(vals))
eval(avg)

ERROR: Error in mean(vals): object 'vals' not found


One of the advantages of non-standard evaluation is that it allows us to evaluate an expression within a different context than the one in which the expression appears lexically.

In [23]:
e <- new.env()
e$vals <- runif(10)

eval(avg, e)

What this means is 

In [33]:
print_func <- function(expr) {
    paste("The value of", deparse(substitute(expr)), "is", expr)
}

x <- 3
y <- 6
print_func(x + y)

[1] "The value of x + y is 9"